https://docs.julialang.org/en/release-0.4/manual/unicode-input/

## DTW


In [1]:
δ(a,b) = (a - b)^2
# first(x) = x[1] firstは元からあるのでいらない
second(x) = x[2]

function minVal(v₁, v₂, v₃)
#    if first(v₁) ≦ minimum([first(v₂), first(v₃)])
    if first(v₁) <= minimum([first(v₂), first(v₃)])
        return v₁, 1
    elseif first(v₂) <= first(v₃)
        return v₂, 2
    else
        return v₃, 3
    end
end

function DTW(A, B)
    S = length(A)
    T = length(B)
    m = Matrix(S, T)
    m[1, 1] = [δ(A[1], B[1]), (0,0)]
    for i in 2:S
        m[i,1] = [m[i-1, 1][1] + δ(A[i], B[1]), [i-1, 1]]
    end
    for j in 2:T
        m[1,j] = [m[1, j-1][1] + δ(A[1], B[j]), [1, j-1]]
    end
    for i in 2:S
        for j in 2:T
            min, index = minVal(m[i-1,j], m[i,j-1], m[i-1,j-1])
            indexes = [[i-1, j], [i, j-1], [i-1, j-1]]
            m[i,j] = first(min) + δ(A[i],B[j]), indexes[index]
        end
    end
    return m
end

DTW (generic function with 1 method)

In [2]:
T = 150
A = sin((1:T)/10)
B = sin((1:T)/10 + pi*0.4)
C = [0 for i in 1:T]

150-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [3]:
DTW(A, B)[150,150]

(9.3585894670215,[150,149])

In [4]:
DTW(A, C)[150,150]

77.67327710659531

## Stochastic Block Model

In [1]:
using Einsum
using DataFrames
using CSV
using StatsBase

In [2]:
K = 8
α = 6
a₀ = b₀ = 0.5

α₁ = transpose(ones(K) * α) # =α₂ 
logΓ = lgamma

lgamma (generic function with 13 methods)

In [3]:
function onehot(i, K)
    ret = zeros(K)
    ret[i] = 1
    return ret
end

function m(x)
    return sum(x,1)
end

m (generic function with 1 method)

In [4]:
# 𝕀 # \BbbI 

LoadError: UndefVarError: 𝕀 not defined

In [36]:
function update_z₁(X, 𝕀z₁, 𝕀z₂)
    N₁, N₂ = size(X)
    m₁ = m(𝕀z₁)
    
    for i in 1:N₁
        @einsum n⁺[k,l] := X[i,j] * 𝕀z₁[i,k] * 𝕀z₂[j,l]
        @einsum n⁻[k,l] := (ones(X)[i,j] - X[i,j]) * 𝕀z₁[i,k] * 𝕀z₂[j,l]
    
        m̂₁ = m(𝕀z₁) - transpose(𝕀z₁[i,:])
        @einsum Σ⁺x𝕀z₂[i,l] := X[i,j] * 𝕀z₂[j,l]
        @einsum Σ⁻x𝕀z₂[i,l] := (ones(X)[i,j] - X[i,j]) * 𝕀z₂[j,l]
        @einsum n̂⁺[k,l] := n⁺[k,l] - 𝕀z₁[i,k] * Σ⁺x𝕀z₂[i,l]
        @einsum n̂⁻[k,l] := n⁻[k,l] - 𝕀z₁[i,k] * Σ⁻x𝕀z₂[i,l]
        
        α̂₁ = α₁ + m̂₁
        â = a₀ + n̂⁺
        b̂ = b₀ + n̂⁻
        
        temp⁺ = zeros(â)
        temp⁻ = zeros(â)
        temp = zeros(â)
        for j in 1:size(temp⁺)[1]
            temp⁺[j,:] = Σ⁺x𝕀z₂[i,:]
            temp⁻[j,:] = Σ⁻x𝕀z₂[i,:]
            temp[j,:] = sum(𝕀z₂,1)
        end
        
        @einsum p_z₁[k,l] := exp(logΓ(â + b̂)-logΓ(â)-logΓ(b̂) 
            + logΓ(â+temp⁺)+logΓ(b̂+temp⁻)-logΓ(â+b̂+temp))[k,l]
        p_z₁ = α̂₁ .* transpose(prod(p_z₁, 2))
        p_z₁ /= sum(p_z₁)
        
        𝕀z₁[i,:] = onehot(sample(1:K, Weights(p_z₁[:])), K)
    end
    return 𝕀z₁
end

update_z₁ (generic function with 1 method)

In [16]:
data = readtable("./bi_data.csv")
X = hcat(data.columns...)

120×120 DataArrays.DataArray{Int64,2}:
 0  0  1  0  0  0  0  1  0  1  1  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  1  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  1  0  1  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  1  0  0  0  0  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  1  0  0  0  0  0  0  0  0  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  1  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  1  1  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  

In [24]:
𝕀z₁ = zeros(size(X)[1],K)
𝕀z₁[:,1] = 1
𝕀z₂ = zeros(size(X)[1],K)
𝕀z₂[:,1] = 1

1

In [37]:
𝕀z₁ = update_z₁(X, 𝕀z₁, 𝕀z₂)
𝕀z₂ = update_z₁(transpose(X), 𝕀z₂, 𝕀z₁)

[37.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 37.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 37.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 37.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 37.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 37.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 37.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 37.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[7.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 7.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 7.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 7.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 7.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 7.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 7.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 7.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[6.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 6.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 6.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 6.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 6.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 6.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 6.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 6.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[13.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 13.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 13.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 13.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 13.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 13.0 0.0 0.0 0.0 0.0 0.0 0

LoadError: InterruptException:

In [14]:
mxval, mxindx = findmax(𝕀z₂, 2)
for i in ind2sub(size(𝕀z₂), vec(mxindx))[2]
    print(i)
end

818111181818111881181115118111151881116211111171145111118151811871811111818112111111111111111111111171111141111111111111

関数直した時に，その関数を読んでる関数ももう一度実行しないといけない．